In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.auto import tqdm

from scipy.stats import wilcoxon
import warnings


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
exchange = "cboe"
models = ["gbm", "classical"] # ["gbm", "classical"]
subset = "test" # "all"
strategy = "supervised"  # "transfer"


In [3]:
key = f"{exchange}_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact cboe_supervised_raw:latest, 1937.19MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.1
wandb: Downloading large artifact cboe_gbm_supervised_test:latest, 56.26MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_classical_supervised_test:latest, 212.93MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )


results = []
for i, model in tqdm(enumerate(models)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=models)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:02,  1.03s/it]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        ("gbm", "gbm(classical-size)"),
        ("gbm", "gbm(ml)"),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [6]:
results_data


gbm                             classical            \
         gbm(classical) gbm(classical-size) gbm(ml)  tick(ex) quote(ex)   
index                                                                     
29930521              1                   1       1       1.0      -1.0   
29894717              1                   1       1      -1.0       1.0   
29909660              1                   1       1      -1.0      -1.0   
29939322             -1                  -1      -1      -1.0      -1.0   
29883818             -1                   1       1      -1.0       1.0   
...                 ...                 ...     ...       ...       ...   
37155409              1                  -1      -1       1.0      -1.0   
37155410              1                   1       1       1.0       1.0   
37104659             -1                  -1      -1      -1.0       1.0   
37108365              1                   1       1      -1.0      -1.0   
37155411              1                   1       1      -1.0      -1.0   

                                  \
         lr(ex) emo(ex) clnv(ex)   
index                              
29930521   -1.0    -1.0     -1.0   
29894717    1.0     1.0      1.0   
29909660   -1.0    -1.0     -1.0   
29939322   -1.0     1.0     -1.0   
29883818    1.0     1.0      1.0   
...         ...     ...      ...   
37155409   -1.0    -1.0     -1.0   
37155410    1.0     1.0      1.0   
37104659    1.0     1.0      1.0   
37108365   -1.0    -1.0     -1.0   
37155411   -1.0    -1.0     -1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
index                                                                                   
29930521                                                1.0                             
29894717                                                1.0                             
29909660                                               -1.0                             
29939322                                               -1.0                             
29883818                                                1.0                             
...                                                     ...                             
37155409                                               -1.0                             
37155410                                                1.0                             
37104659                                               -1.0                             
37108365                                                1.0                             
37155411                                                1.0                             

[7431766 rows x 9 columns]

### Robustness Checks

In [7]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        ((mid - half_spread) < trade_price) & (trade_price < (mid + half_spread)),
        1,  # inside
        np.where(
            (trade_price == ask) | (bid == trade_price),
            2,  # at quotes
            np.where(
                (trade_price < (mid - half_spread))
                | ((mid + half_spread) > trade_price),
                3,
                4,
            ),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print["all"] = "all"


X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [8]:
X_print = pd.concat([X_print, results_data], axis=1)


In [9]:
X_print.head().T


index,29930521,29894717,29909660,29939322,29883818
buy_sell,1,1,1,-1,1
TRADE_PRICE,0.02,0.27,0.2,0.12,1.85
OPTION_TYPE,C,P,C,C,C
issue_type,Others,Stock option,Stock option,Others,Stock option
TRADE_SIZE_binned,"(0,1]","(5,11]","(0,1]","(0,1]","(3,5]"
year_binned,2016,2016,2016,2016,2016
ttm_binned,<= 1,<= 1,(1-2],<= 1,(1-2]
myn_binned,(0.7-0.9],(0.9-1.1],(0.7-0.9],(0.9-1.1],(0.9-1.1]
prox_q_binned,at quotes,inside,inside,inside,at quotes
mid,0.025,0.225,0.35,0.135,1.725


## Results Set Generation

In [10]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    # "(": "(",  # put interval start in math env
    # "]": "]",  # put interval end in math env
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Proximity to Quote",
    "all":"All Trades",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [11]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)
    res = (res.apply(
            highlight_max, props="font-weight:bold;", axis=bold_axis
        )
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        # .hide(axis="index", level=0)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [12]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


In [13]:
results_data.columns.tolist()


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [14]:
X_print.head()

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,all,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,,
29930521,1,0.02,C,Others,"(0,1]",2016,<= 1,(0.7-0.9],at quotes,0.025,all,1,1,1,1.0,-1.0,-1.0,-1.0,-1.0,1.0
29894717,1,0.27,P,Stock option,"(5,11]",2016,<= 1,(0.9-1.1],inside,0.225,all,1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0
29909660,1,0.20,C,Stock option,"(0,1]",2016,(1-2],(0.7-0.9],inside,0.350,all,1,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
29939322,-1,0.12,C,Others,"(0,1]",2016,<= 1,(0.9-1.1],inside,0.135,all,-1,-1,-1,-1.0,-1.0,-1.0,1.0,-1.0,-1.0
29883818,1,1.85,C,Stock option,"(3,5]",2016,(1-2],(0.9-1.1],at quotes,1.725,all,-1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0


## Accuracy Calculation

In [15]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        # acc_tot = accuracy_score(
        #     X_print["buy_sell"].astype("int8"), X_print[classifier]
        # )

        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        # res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 8/8 [01:21<00:00, 10.13s/it]


In [16]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [17]:
master


gbm                                 \
                                gbm(classical) gbm(classical-size)    gbm(ml)   
Option Type        C                 69.929332           74.756527  77.082873   
                   P                 70.760558           74.645476  76.864945   
Security Type      Index option      68.338618           72.520754  76.430684   
                   Others            71.493258           75.251770  77.361457   
                   Stock option      69.979251           74.662893  76.871719   
Trade Size         (0,1]             68.092407           72.620323  75.332737   
                   (1,3]             69.205331           73.683006  76.167690   
                   (3,5]             69.687525           74.403358  76.599365   
                   (5,11]            70.951622           75.010330  77.095774   
                   >11               74.590981           78.800083  80.454838   
Year               2016              70.843835           75.275833  77.831739   
                   2017              70.162344           74.549874  76.751275   
Time to Maturity   <= 1              70.555488           74.287969  76.510357   
                   (1-2]             71.600389           75.493369  77.917123   
                   (2-3]             71.099750           75.899742  78.030617   
                   (3-6]             70.271222           76.207334  78.289240   
                   (6-12]            69.303696           74.826243  77.438567   
                   > 12              64.125740           72.253504  75.045868   
Moneyness          <= 0.7            71.619038           77.268103  79.809461   
                   (0.7-0.9]         71.104891           75.173820  77.717153   
                   (0.9-1.1]         70.844434           74.973394  77.064097   
                   (1.1-1.3]         63.309037           70.194063  72.987062   
                   > 1.3             63.025877           69.954130  74.292110   
Proximity to Quote at mid            62.321390           67.947788  70.502941   
                   inside            70.691264           73.931190  75.874193   
                   at quotes         73.892075           83.785245  87.718462   
                   outside           72.916667           74.382716  70.987654   
                   unknown           79.216315           79.491292  80.052704   
All Trades         all               70.308753           74.705837  76.983398   

                                 classical                                   \
                                  tick(ex)  quote(ex)     lr(ex)    emo(ex)   
Option Type        C             48.955333  60.850513  60.553543  48.243383   
                   P             49.490788  60.934859  60.747611  48.077527   
Security Type      Index option  48.487310  50.334319  50.260448  39.699766   
                   Others        49.153068  65.261752  64.922796  50.091092   
                   Stock option  49.280091  59.952437  59.729468  48.081917   
Trade Size         (0,1]         49.041394  58.033645  57.900579  46.664499   
                   (1,3]         49.214207  61.326282  61.092161  48.401654   
                   (3,5]         48.820721  61.082123  60.806340  48.227728   
                   (5,11]        49.106302  61.512827  61.232605  48.207533   
                   >11           49.771205  63.820646  63.446330  50.008040   
Year               2016          49.402674  60.626287  60.445217  47.905382   
                   2017          49.144221  60.960901  60.696006  48.239445   
Time to Maturity   <= 1          49.114442  61.777120  61.463927  48.957128   
                   (1-2]         49.712812  61.422130  61.261946  48.692573   
                   (2-3]         49.331157  61.072769  60.865919  47.269492   
                   (3-6]         49.013532  60.104698  59.909792  46.616888   
                   (6-12]        49.364264  59.128210  59.023008  46.645915   
                   > 12       

In [18]:
master.style.pipe(
    set_tex_style, caption=("master-long", "master-short"), label=f"{key}-master", bold_axis=0
)


## Effective Spread 🚧

In [19]:
classifiers


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [20]:
eff_dfs = []


def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = (
        effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")
        * 100
    )

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")

    return pd.Series(
        {
            "nominal": nom,
            "rel": rel,
            # 'statistic':wilcoxon_res.statistic,
            # 'pvalue':wilcoxon_res.pvalue
        }
    )


for criterion in tqdm(criterions):
    results = []

    for classifier in tqdm(classifiers):
        res = X_print.groupby([criterion])[
            ["TRADE_PRICE", "mid", classifier, "buy_sell"]
        ].apply(stats, classifier)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}-eff-spread",
    )

    # store all result sets for later use
    eff_dfs.append(result_df)


100%|██████████| 8/8 [00:43<00:00,  5.48s/it]


In [21]:
master_eff_spread = pd.concat(eff_dfs, axis=1, keys=LUT_INDEX.values()).T


In [22]:
master_eff_spread


gbm                 \
                                gbm(classical)                  
                                       nominal            rel   
Option Type        C                  0.037183  5.568612e+302   
                   P                  0.023943  -1.961121e+01   
Security Type      Index option      -0.008027   1.692972e+00   
                   Others             0.024162   6.773617e+00   
                   Stock option       0.037412  9.112811e+302   
Trade Size         (0,1]              0.027974  2.176855e+303   
                   (1,3]              0.032963   1.692723e+00   
                   (3,5]              0.032180   5.789902e+00   
                   (5,11]             0.030742   6.143302e+00   
                   >11                0.033330   7.227341e+00   
Year               2016               0.028062   4.682052e+00   
                   2017               0.031980   6.174744e+00   
Time to Maturity   <= 1               0.026884   7.550903e+00   
                   (1-2]              0.032591   5.271263e+00   
                   (2-3]              0.025268  4.398703e+303   
                   (3-6]              0.036415   2.784819e+00   
                   (6-12]             0.036176   2.444805e+00   
                   > 12               0.069044  6.471238e+303   
Moneyness          <= 0.7             0.006647   9.450880e+00   
                   (0.7-0.9]          0.020980   9.551971e+00   
                   (0.9-1.1]          0.032613  4.306566e+302   
                   (1.1-1.3]          0.047263   8.209339e-01   
                   > 1.3              0.066832   3.686985e-01   
Proximity to Quote at mid             0.000000   0.000000e+00   
                   inside             0.059250   6.556484e+00   
                   at quotes         -0.095599   6.135748e+00   
                   outside           -5.327608   1.664593e+01   
                   unknown            0.519734            inf   
All Trades         all                0.031139  3.026470e+302   

                                                                               \
                                gbm(classical-size)                   gbm(ml)   
                                            nominal            rel    nominal   
Option Type        C                       0.021744  5.568612e+302   0.016731   
                   P                       0.013383  -2.088343e+01   0.010697   
Security Type      Index option            0.001682   2.497100e+00  -0.015639   
                   Others                  0.011002   5.665873e+00   0.013227   
                   Stock option            0.022223  9.112811e+302   0.016818   
Trade Size         (0,1]                   0.005834  2.176855e+303   0.003788   
                   (1,3]                   0.022711   3.988364e-01   0.019273   
                   (3,5]                   0.025546   4.746889e+00   0.018529   
                   (5,11]                  0.020257   5.290992e+00   0.014999   
                   >11                     0.022527   6.695074e+00   0.018656   
Year               2016                    0.009825   2.982258e+00   0.001754   
                   2017                    0.020144   4.983556e+00   0.017320   
Time to Maturity   <= 1                    0.017995   6.051896e+00   0.013382   
                   (1-2]                   0.018591   4.190161e+00   0.017595   
                   (2-3]                   0.006083  4.398703e+303   0.012195   
                   (3-6]                   0.019798   1.833091e+00   0.021648   
                   (6-12]                  0.022991   1.339052e+00   0.019827   
                   > 12                    0.021152  6.471238e+303  -0.012343   
Moneyness          <= 0.7                 -0.006295   6.665789e+00  -0.020613   
                   (0.7-0.9]               0.008382   7.357694e+00   0.002450   
                   (0.9-1.1]               0.021276  4.306566e+302   0.021431   
                  

In [23]:
master_eff_spread.style.pipe(
    set_tex_style,
    caption=("master-short", "master-long"),
    label=f"{key}-master-eff-spread",
    bold_axis=0,
)


## Change in Parenthesis 🅾️

```latex
# https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
\begin{table}
    \centering
    \caption{test of combination with change}
    \label{tab:combo}
    \begin{tabular}{lSSSSSSSS}
        \toprule
        {} & \multicolumn{2}{l}{Index option} & \multicolumn{2}{l}{Others} & \multicolumn{2}{l}{Stock options} & \multicolumn{2}{l}{all} \\
        \midrule
        classical-size & 1.0 & \parl-56.42\parr & 2.0 & \parl-74.35\parr & -73.5 & \parl-143.93\parr & 5.0 & \parl-67.33\parr \\
        \bottomrule
        \end{tabular}
\end{table}
```

In [24]:
master

gbm                                 \
                                gbm(classical) gbm(classical-size)    gbm(ml)   
Option Type        C                 69.929332           74.756527  77.082873   
                   P                 70.760558           74.645476  76.864945   
Security Type      Index option      68.338618           72.520754  76.430684   
                   Others            71.493258           75.251770  77.361457   
                   Stock option      69.979251           74.662893  76.871719   
Trade Size         (0,1]             68.092407           72.620323  75.332737   
                   (1,3]             69.205331           73.683006  76.167690   
                   (3,5]             69.687525           74.403358  76.599365   
                   (5,11]            70.951622           75.010330  77.095774   
                   >11               74.590981           78.800083  80.454838   
Year               2016              70.843835           75.275833  77.831739   
                   2017              70.162344           74.549874  76.751275   
Time to Maturity   <= 1              70.555488           74.287969  76.510357   
                   (1-2]             71.600389           75.493369  77.917123   
                   (2-3]             71.099750           75.899742  78.030617   
                   (3-6]             70.271222           76.207334  78.289240   
                   (6-12]            69.303696           74.826243  77.438567   
                   > 12              64.125740           72.253504  75.045868   
Moneyness          <= 0.7            71.619038           77.268103  79.809461   
                   (0.7-0.9]         71.104891           75.173820  77.717153   
                   (0.9-1.1]         70.844434           74.973394  77.064097   
                   (1.1-1.3]         63.309037           70.194063  72.987062   
                   > 1.3             63.025877           69.954130  74.292110   
Proximity to Quote at mid            62.321390           67.947788  70.502941   
                   inside            70.691264           73.931190  75.874193   
                   at quotes         73.892075           83.785245  87.718462   
                   outside           72.916667           74.382716  70.987654   
                   unknown           79.216315           79.491292  80.052704   
All Trades         all               70.308753           74.705837  76.983398   

                                 classical                                   \
                                  tick(ex)  quote(ex)     lr(ex)    emo(ex)   
Option Type        C             48.955333  60.850513  60.553543  48.243383   
                   P             49.490788  60.934859  60.747611  48.077527   
Security Type      Index option  48.487310  50.334319  50.260448  39.699766   
                   Others        49.153068  65.261752  64.922796  50.091092   
                   Stock option  49.280091  59.952437  59.729468  48.081917   
Trade Size         (0,1]         49.041394  58.033645  57.900579  46.664499   
                   (1,3]         49.214207  61.326282  61.092161  48.401654   
                   (3,5]         48.820721  61.082123  60.806340  48.227728   
                   (5,11]        49.106302  61.512827  61.232605  48.207533   
                   >11           49.771205  63.820646  63.446330  50.008040   
Year               2016          49.402674  60.626287  60.445217  47.905382   
                   2017          49.144221  60.960901  60.696006  48.239445   
Time to Maturity   <= 1          49.114442  61.777120  61.463927  48.957128   
                   (1-2]         49.712812  61.422130  61.261946  48.692573   
                   (2-3]         49.331157  61.072769  60.865919  47.269492   
                   (3-6]         49.013532  60.104698  59.909792  46.616888   
                   (6-12]        49.364264  59.128210  59.023008  46.645915   
                   > 12       

In [25]:
base = master[[("classical","quote(ex)"),("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)"), ("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)")]]
revised = master[[("gbm","gbm(classical)"),("gbm","gbm(classical-size)"), ("gbm", "gbm(ml)")]]

In [29]:

def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    
    
    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (combo[mul_col] - base.iloc[:,i]).round(2)
        # print(combo.shape)
        # # define custom brackets that are not parsed by sunitx
        # combo.iloc[:,[(mul_col[0], "pm")]] = 
        # # sort to group together columns
        # combo.sort_index(axis=1, inplace=True)
        # cols.append(diff)
        combo.sort_index(axis=1, inplace=True)
        
    return combo


In [30]:
diff = combine_results(revised, base)

diff.style.to_latex(
            f"diff-{key}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label=f"tab:diff-{key}",
            caption=(f"long-diff-{key}", f"short-diff-{key}"),
            convert_css=True,
        )

In [31]:
diff


gbm(classical)        gbm(classical-size)  \
                                           nom     pm                 nom   
Option Type        C                 69.929332   9.08           74.756527   
                   P                 70.760558   9.83           74.645476   
Security Type      Index option      68.338618  18.00           72.520754   
                   Others            71.493258   6.23           75.251770   
                   Stock option      69.979251  10.03           74.662893   
Trade Size         (0,1]             68.092407  10.06           72.620323   
                   (1,3]             69.205331   7.88           73.683006   
                   (3,5]             69.687525   8.61           74.403358   
                   (5,11]            70.951622   9.44           75.010330   
                   >11               74.590981  10.77           78.800083   
Year               2016              70.843835  10.22           75.275833   
                   2017              70.162344   9.20           74.549874   
Time to Maturity   <= 1              70.555488   8.78           74.287969   
                   (1-2]             71.600389  10.18           75.493369   
                   (2-3]             71.099750  10.03           75.899742   
                   (3-6]             70.271222  10.17           76.207334   
                   (6-12]            69.303696  10.18           74.826243   
                   > 12              64.125740  11.62           72.253504   
Moneyness          <= 0.7            71.619038  16.74           77.268103   
                   (0.7-0.9]         71.104891  12.06           75.173820   
                   (0.9-1.1]         70.844434   8.46           74.973394   
                   (1.1-1.3]         63.309037   7.79           70.194063   
                   > 1.3             63.025877  10.63           69.954130   
Proximity to Quote at mid            62.321390  12.49           67.947788   
                   inside            70.691264   3.75           73.931190   
                   at quotes         73.892075  38.22           83.785245   
                   outside           72.916667  -6.87           74.382716   
                   unknown           79.216315  20.38           79.491292   
All Trades         all               70.308753   9.42           74.705837   

                                          gbm(ml)         
                                    pm        nom     pm  
Option Type        C              8.64  77.082873  10.97  
                   P              7.63  76.864945   9.85  
Security Type      Index option   4.84  76.430684   8.75  
                   Others         6.69  77.361457   8.80  
                   Stock option   9.09  76.871719  11.30  
Trade Size         (0,1]          8.55  75.332737  11.26  
                   (1,3]          8.32  76.167690  10.80  
                   (3,5]          8.09  76.599365  10.28  
                   (5,11]         8.65  77.095774  10.73  
                   >11            7.12  80.454838   8.78  
Year               2016          10.81  77.831739  13.37  
                   2017           7.46  76.751275   9.66  
Time to Maturity   <= 1           7.67  76.510357   9.89  
                   (1-2]          8.49  77.917123  10.92  
                   (2-3]          8.31  78.030617  10.44  
                   (3-6]          8.64  78.289240  10.72  
                   (6-12]         8.26  77.438567  10.87  
                   > 12          12.32  75.045868  15.11  
Moneyness          <= 0.7         7.84  79.809461  10.38  
                   (0.7-0.9]      8.09  77.717153  10.63  
                   (0.9-1.1]      7.96  77.064097  10.05  
                   (1.1-1.3]     10.25  72.987062  13.04  
                   > 1.3         11.22  74.292110  15.55  
Proximity to Quote at mid         7.71  70.502941  10.27  
                   inside         8.10  75.874193  10.05  
                   at quotes      9.02  87.718462  